In [2]:
import sys
sys.path.append("../../")
from Scripts import *

In [ ]:
results_dict = {}
cost_curve_dicts = {'Optimization': {}, 'Planning': {}}

opt_counter = 0
plan_counter = 0
for n_sats in range(3,13,1):
    results_dict[n_sats] = {'Optimization': {'Total Results': pd.DataFrame({'Number of Targets': [], 'Avg_Percentage': [], 'Max_Time': [], 'Avg_Time': [], 'Std_Time': []})}, 'Planning': pd.DataFrame({'Total Results': {'Number of Targets': [], 'Avg_Percentage': [], 'Max_Time': [], 'Avg_Time': [], 'Std_Time': []}})}


    if os.path.exists(f'../../Pop_Over_Gen/{n_sats}.csv'):
        df = pd.read_csv(f'../../Pop_Over_Gen/{n_sats}.csv')
        results_dict[n_sats]['Pop_Over_Gen'] = df


    for n_targets in [15,34,65,82,109,186,494,1131]:
        if os.path.exists(f'../../Optimization_Results/{n_sats}_{n_targets}.csv'):
            df = pd.read_csv(f'../../Optimization_Results/{n_sats}_{n_targets}.csv').round(4)
            results_dict[n_sats]['Optimization'][n_targets] = df
            results_dict[n_sats]['Optimization']['Total Results'].loc[len(results_dict[n_sats]['Optimization']['Total Results'])] = [df["Unplanned (%)"].count(), df["Unplanned (%)"].mean().round(4), df["Unplanned (Time)"].max().round(1),  df["Unplanned (Time)"].mean().round(1), df["Unplanned (Time)"].std().round(1)]
            opt_counter += 1

        # if os.path.exists(f'../../Planning_Results/{n_sats}_{n_targets}.csv'):
        #     df = pd.read_csv(f'../../Planning_Results/{n_sats}_{n_targets}.csv').round(4)
        #     results_dict[n_sats]['Planning'][n_targets] = df
        #     results_dict[n_sats]['Planning']['Total Results'].loc[len(results_dict[n_sats]['Planning']['Total Results'])] = [df["Planned (%)"].count(), df["Planned (%)"].mean().round(4), df["Planned (Time)"].max().round(1), df["Planned (Time)"].mean().round(1), df["Planned (Time)"].std().round(1)]
        #     plan_counter += 1

    cost_curve_dicts['Optimization'][n_sats] = results_dict[n_sats]['Optimization']['Total Results'][results_dict[n_sats]['Optimization']['Total Results']['Avg_Percentage'] == 100]
    if len(results_dict[n_sats]['Planning']['Total Results']['Number of Targets']) != 0:
        cost_curve_dicts['Planning'][n_sats] = results_dict[n_sats]['Planning']['Total Results'][results_dict[n_sats]['Planning']['Total Results']['Avg_Percentage'] == 100]


    

# axis's = Number of Targets, Avg_Time, Max_Time, Std_Time
# curve_type = Optimzation, Planning
Generate_Performance_Curve(cost_curve_dicts,curve_type='Optimization', xaxis='Number of Targets', yaxis='Max_Time') 
# Generate_Performance_Curve(cost_curve_dicts,curve_type='Planning', xaxis='Number of Targets', yaxis='Avg_Time') 

In [ ]:
def pop_over_gen_plotter_animator(n_sats, results_dict, hover_text=['Alt', 'Inc', 'Initial_Raan', 'Delta_Raan','Num_Planes', 'Avg_Percentage', 'Std_Percentage', 'Avg_Time',
       'Std_Time'],plot_title='Satellites Pop_Over_Gen', xaxis='Avg_Time', yaxis='Avg_Percentage', graph=True, animation=True, constraint=True, xticks = [0, 10, 20, 30, 40], yticks=[0, 25, 50, 75, 100]):

    pop_over_gen_df = results_dict[n_sats]['Pop_Over_Gen']
    colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan', 'olive', 'yellow']

    figs = []
    if graph:
        fig_graph = make_subplots()
        figs.append(fig_graph)
        
    if animation:
        fig_animation = make_subplots()
        figs.append(fig_animation)
    

    frames = []

    df = pop_over_gen_df[pop_over_gen_df['Gen'] == 0]

    scatter = go.Scatter(
        x=df[xaxis],
        y=df[yaxis],
        hovertext=df[hover_text].astype(str).agg(', '.join, axis=1),
        hoverinfo='text',
        mode='markers',
        name=f'{0}',
        marker=dict(color=colors[0], size=8),
    )

    

    for i, gen in enumerate(pop_over_gen_df['Gen'].unique()):
        df = pop_over_gen_df[pop_over_gen_df['Gen'] == gen]


        scatter = go.Scatter(
            x=df[xaxis],
            y=df[yaxis],
            hovertext=df[hover_text].astype(str).agg(', '.join, axis=1),
            hoverinfo='text',
            mode='markers',
            name=f'{gen}',
            marker=dict(color=colors[i % len(colors)], size=8),
        )
        
        if i == 0 and animation:
            fig_animation.add_trace(scatter)

        if graph:
            fig_graph.add_trace(scatter)

        frames.append(go.Frame(data=[scatter], name=str(gen)))

    for fig in figs:

        if constraint:
            fig.add_vline(30, line_dash='dash', line_color='red')
            fig.add_hline(100, line_dash='dash', line_color='red')

            fig.add_annotation(x=30.5, y=50, text='30 Day Constraint', font=dict(color='red', size=15), textangle=90, showarrow=False)
            fig.add_annotation(x=15, y=105, text='100% Imaged Constraint', font=dict(color='red', size=15), showarrow=False)


        if graph and fig == fig_graph:
            fig.update_layout(
                title=f'{plot_title}',
                xaxis_title=xaxis,
                yaxis_title=yaxis,
                xaxis=dict(tickmode='array',tickvals=xticks),
                yaxis=dict(tickmode='array',tickvals=yticks, range=[yticks[0]-10, yticks[-1]+10]),
                legend=dict(title='Gen',title_font=dict(size=15),font=dict(size=12, color='black')),
                font=dict(size=15))

        if animation and fig == fig_animation:
            
            fig.update_layout(title=f'{plot_title}',
            xaxis_title=xaxis,
            yaxis_title=yaxis,
            xaxis=dict(tickmode='array',tickvals=xticks),
            yaxis=dict(tickmode='array',tickvals=yticks,range=[yticks[0]-10, yticks[-1]+10]),
            legend=dict(title='Gen',title_font=dict(size=15),font=dict(size=12, color='black')),
            font=dict(size=15), 
            updatemenus=[{
            'buttons': [
                {
                    'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}],
                    'label': 'Play',
                    'method': 'animate'
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                    'label': 'Pause',
                    'method': 'animate'
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }],
        sliders=[{
            'active': 0,
            'yanchor': 'top',
            'xanchor': 'left',
            'currentvalue': {
                'font': {'size': 20},
                'prefix': 'Generation: ',
                'visible': True,
                'xanchor': 'right'
            },
            'transition': {'duration': 300, 'easing': 'cubic-in-out'},
            'pad': {'b': 10, 't': 50},
            'len': 0.9,
            'x': 0.1,
            'y': 0,
            'steps': [
                {
                    'args': [[str(gen)], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 300}}],
                    'label': str(gen),
                    'method': 'animate'
                } for gen in pop_over_gen_df['Gen'].unique()
            ]
        }])
            fig.frames = frames
        fig.show()

n_sats=5
pop_over_gen_plotter_animator(n_sats, results_dict, graph=True, animation=True, xaxis='Avg_Time', yaxis='Avg_Percentage',plot_title=f'{n_sats} Satellite Constellation Populations Over Generations')


In [103]:
def create_pareto(df, dvs, plot_title, objective1='Cost', objective2='Avg_Percentage', pareto = True, color_scale='Run_Num'):

    df = df.sort_values(by=objective1, ignore_index = True)
    df[objective2] = df[objective2]

    dvs_ovs = dvs
    dvs_ovs.append(objective1)
    dvs_ovs.append(objective2)

    fig = make_subplots()

    # Scatter plot of all data points
    scatter = go.Scatter(
        x=df[objective1],
        y=df[objective2],
        hovertext=df[dvs_ovs].astype(str).agg(', '.join, axis=1),
        hoverinfo='text',
        mode='markers',
        name='Data Points',
        marker=dict(
        size=8,
        color=df[color_scale],
        colorscale='Viridis',  # You can choose a different colorscale if desired
        colorbar=dict(title=color_scale)))
    fig.add_trace(scatter)

    # Line plot of the Pareto frontier
    if pareto:
        pareto_frontier = []
        pareto_frontier = [[df[idx][0] for idx in dvs_ovs]]

        for index, row in df[1:].iterrows():
            point = [row[idx] for idx in dvs_ovs]
            if point[-1] < pareto_frontier[-1][-1]:
                pareto_frontier.append(point)

        pareto_frontier = pd.DataFrame(pareto_frontier, columns=dvs_ovs)
        pareto_line = go.Scatter(
            x=pareto_frontier[objective1],
            y=pareto_frontier[objective2],
            hovertext=pareto_frontier[dvs_ovs].astype(str).agg(', '.join, axis=1),
            hoverinfo='text',
            mode='lines+markers',
            name='Pareto Frontier',
            line=dict(color='green'),
            marker=dict(size=8))
        fig.add_trace(pareto_line)
    
    


    # Update layout
    fig.update_layout(
        title=f'{plot_title}',
        xaxis_title=f'{objective1}',
        yaxis_title=f'{objective2}',
        legend=dict(x=1, y=1.25),
        template='plotly_white'
    )

    # Show the figure
    fig.show()
    return pareto_frontier

In [132]:
df = pd.read_csv('../../Pop_Over_Gen/pop_gen.csv')
df['Avg_Percentage'] = df['Avg_Percentage'].round(4)
df = df[(df['Cost'] < 50)]
df = df[(df['Avg_Percentage'] >= 25)]


In [133]:
create_pareto(df, dvs='Alt,Inc,Initial_Raan,Delta_Raan,Num_Sats,Num_Planes'.split(','),objective2='Avg_Percentage',plot_title='Pareto Frontier',pareto=True)

,Alt,Inc,Initial_Raan,Delta_Raan,Num_Sats,Num_Planes,Cost,Avg_Percentage
0,610.0,93.0,81.0,17.0,10.0,7.0,17.0,25.2881
1,629.0,93.0,63.0,15.0,10.0,8.0,18.0,25.2675


In [91]:
unique_rows = df['Alt,Inc,Initial_Raan,Delta_Raan,Num_Sats,Num_Planes,Avg_Percentage,Cost'.split(',')].drop_duplicates()
unique_rows.sort_values(by='Cost')


,Alt,Inc,Initial_Raan,Delta_Raan,Num_Sats,Num_Planes,Avg_Percentage,Cost
1345,623,93,41,5,9,7,100.0,16.0
1153,623,93,133,5,9,7,100.0,16.0
95,626,94,147,4,9,9,100.0,18.0
1136,623,93,133,5,9,9,100.0,18.0
76,626,94,147,4,10,9,100.0,19.0
340,612,93,86,4,11,10,100.0,21.0
1208,584,93,95,5,11,11,100.0,22.0
